# Detection of Toxic Comments

Kaggle competition: [Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge)

In [101]:
seed = 333

## Import

## Load Data

In [102]:
import csv
from datetime import datetime
import string
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation,GlobalMaxPool1D,Bidirectional
from keras.layers.embeddings import Embedding
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk import word_tokenize

In [103]:
cache_english_stopwords=stopwords.words('english')

def data_clean(message):
    temp_tw_list = word_tokenize(message)
    # Remove stopwords
    list_no_stopwords=[i for i in temp_tw_list if i.lower() not in     cache_english_stopwords]
    # Remove hyperlinks
    list_no_hyperlinks=[re.sub(r'https?:\/\/.*\/\w*','',i) for i in list_no_stopwords]
    # Remove hashtags
    list_no_hashtags=[re.sub(r'#', '', i) for i in list_no_hyperlinks]
    #Remove numbers
    list_no_numbers = [re.sub(r'$\d+\W+|\b\d+\b|\W+\d+$','', i) for i in list_no_hashtags]
   # Remove Punctuation and split 's, 't, 've with a space for filter
    list_no_punctuation=[re.sub(r'['+string.punctuation+']+', ' ', i) for i in list_no_numbers]
    # Remove multiple whitespace
    new_sent = ' '.join(list_no_punctuation)
    # Remove any words with 2 or fewer letters
    filtered_list = word_tokenize(new_sent)
    list_filtered = [re.sub(r'^\w\w?$', '', i) for i in filtered_list]
    filtered_sent =' '.join(list_filtered)
    clean_sent=re.sub(r'\s\s+', ' ', filtered_sent)
    #Remove any whitespace at the front of the sentence
    clean_sent=clean_sent.lstrip(' ')
    return clean_sent


In [104]:
def read_comments(filename=''):
    comments = []
    labels = []
    ids = []

    with open(filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        header = True
        for row in csvReader:
            if header:
                header = False
                continue
            ids.append(row[0])
            comments.append(data_clean(row[1]))
            if len(row) > 2:
                classes = []
                for i in range(2, 8):
                    classes.append(row[i])
                labels.append(classes)

    ids = np.asarray(ids)
    X = np.asarray(comments)
    Y = np.asarray(labels, dtype=int)

    return (ids, X, Y)

In [ ]:
id_train, X_train, Y_train = read_comments('./data/train.csv')
id_test, X_test, _ = read_comments('./data/test.csv')
labels = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

max_comment = 0
for i in range(X_train.shape[0]):                               
    l = len(X_train[i].split())
    if max_comment < l:
        max_comment = l
for i in range(X_test.shape[0]):                               
    l = len(X_test[i].split())
    if max_comment < l:
        max_comment = l
num_classes = len(labels)

## Split Data

In [ ]:
kfold = KFold(n_splits=5,random_state=seed)
# folds = list()
# for train_index, dev_index in kfold.split(X_train):
#     folds.append({
#         'train_index' : train_index,
#         'dev_index' : dev_index,     
#     })

## Load Embeddings

In [ ]:
def read_embeddings(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [ ]:
word_to_index, index_to_word, word_to_vec = read_embeddings('./glove/glove.twitter.27B.25d.txt')
word_to_vec["0.065581"] = [ 0.39605,  -0.96669,   0.23706,  -0.41379,  -0.97006,   0.16601,  -1.292,
 -0.58989,   0.11632,  -1.365,    -0.27939,  -0.57222,  -0.97108,  -0.56319,
 -0.015263, -0.70465,  -0.13867 ,  1.0702 ,  -0.25557  , 0.25122,  -0.87755,
  0.70999 ,  0.9118 ,  -0.30077, 0 ]


In [ ]:
for r in X_train:
    words = r.split()
    for w in words:
        if w not in word_to_index:
            if 'fuck' in w:
                print(w)



## Custom Loss

In [ ]:
from sklearn.metrics import log_loss
from keras.backend import int_shape
import keras.backend as K
import tensorflow as tf
_EPSILON = K.epsilon()
    
def custom_loss(y_true, y_pred):
    losses = []
    y_pred = K.clip(y_pred, _EPSILON, 1.0-_EPSILON)
    num_classes = int_shape(y_pred)[1]
    for i in range(num_classes):
        losses.append(K.mean(-(
            y_true[:, i]*K.log(y_pred[:, i]) - (1-y_true[:, i])*K.log(1-y_pred[:, i])
        ), axis = -1))
    loss = tf.stack(losses)
    loss = K.mean(loss, axis=-1)
        
    return loss
    #return tf.convert_to_tensor(losses, dtype='float32')

## Cross Validation

In [ ]:
np.random.seed(seed)
estimator_baseline = KerasRegressor(build_fn=baseline_model, nb_epoch=1, batch_size=5, verbose=1)
results = cross_val_score(estimator_baseline, averages, Y_train, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

## RNN

In [ ]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    m = X.shape[0]                                   # number of training examples
    X_indices = np.zeros(shape=(m, max_len))
    for i in range(m):                               
        sentence_words = X[i].lower().split()
        if len(sentence_words) > max_len:
            print ("Too many words:", len(sentence_words))
            continue
        j = 0
        for w in sentence_words:
            if w in word_to_index:
                X_indices[i, j] = word_to_index[w]
            else :
                X_indices[i, j] = word_to_index['word']
            j = j + 1
    return X_indices

In [ ]:
X_train_indices = sentences_to_indices(X_train, word_to_index, max_comment)
X_test_indices = sentences_to_indices(X_test, word_to_index, max_comment)

In [ ]:
def pretrained_embedding_layer(word_to_vec, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 
    
    Arguments:
    word_to_vec -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_len = len(word_to_index) + 1                  
    emb_dim = word_to_vec["cucumber"].shape[0]  
    emb_matrix = np.zeros(shape=(vocab_len, emb_dim))
    
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer

In [ ]:
def rnn_model(input_shape, num_classes, word_to_vec, word_to_index):
    """
    Create the model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec -- dictionary mapping every word in a vocabulary into its n-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary 

    Returns:
    model -- a model instance in Keras
    """
    sentence_indices = Input(shape=input_shape, dtype='float32')
    embedding_layer = pretrained_embedding_layer(word_to_vec, word_to_index)
    embeddings = embedding_layer(sentence_indices)   
    
    X = Bidirectional(LSTM(18, return_sequences = False))(embeddings)
    X = Dropout(rate=0.1)(X)
#     X = LSTM(8, return_sequences = False)(X)
#     X = Dropout(rate=0.2)(X)
    X = Dense(units=50, activation='relu')(X)
    X = Dropout(0.1)(X)
    X = Dense(units=num_classes, activation='sigmoid')(X)
    model = Model(inputs = [sentence_indices], outputs = X)
    
    return model

In [ ]:
model_RNN = rnn_model((max_comment,), num_classes, word_to_vec, word_to_index)
model_RNN.summary()

In [ ]:
model_RNN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_RNN.fit(X_train_indices, Y_train, epochs = 2, batch_size = 128, shuffle=True, verbose=True)

## Testing

In [ ]:
import matplotlib.pyplot as plt

def plot_predictions(probabilities, title):
    fig = plt.figure(figsize=(12, 4))
    fig.suptitle(title)
    plt.autoscale(enable=False, axis='y')
    width = 0.85
    ind = np.arange(len(probabilities))
    plt.bar(ind, probabilities, width=width, color='g')
    plt.xticks(ind, labels)
    
def predict(text, model):
    plot_predictions(model.predict(sentences_to_indices(
        np.asarray([text]),word_to_index, max_comment))[0], text)    

In [ ]:
predict("Do you think you are smart? idiot", model_RNN)

In [ ]:
predict("I love you, my darling", model_RNN)

## Submission

In [ ]:
y = model_RNN.predict(X_test_indices)

In [ ]:
result = pd.concat([
    pd.DataFrame(id_test, columns=['id']), 
    pd.DataFrame(y, columns=labels)], axis=1)
result = result.set_index('id')
result = result.fillna(0.0)
result.to_csv('./result/anton_rnn_{}.csv'.format(datetime.now()), header=True)